<a href="https://colab.research.google.com/github/Henil21/Natural_Language_processing/blob/main/Hindi_summary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.3/519.3 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 5.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.6 MB/s eta 0:00:00


In [ ]:
from datasets import load_dataset

dataset = load_dataset("Someman/hindi-summarization")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/55226 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/13807 [00:00<?, ? examples/s]

Generating valid split:   0%|          | 0/17265 [00:00<?, ? examples/s]

In [ ]:
len(dataset['train'])
dataset_train=dataset['train']
dataset

DatasetDict({
    train: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 55226
    })
    test: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 13807
    })
    valid: Dataset({
        features: ['headline', 'summary', 'article'],
        num_rows: 17265
    })
})

In [ ]:
import random
def show_random(sample=3,seed=42):
    random_index=random.randint(0, len(dataset_train)-5)

    for i in range(sample):
        print("\nHeadline\n",dataset['train']['headline'][random_index+i],"\n")
        print("\nsummary\n",dataset['train']['summary'][random_index+i])
        # print("article\n",dataset['train']['article'][random_index+i])

show_random()


Headline
 राहुल गांधी का सरकार में शामिल होना जरूरी : कृष्णा   


summary
 कैबिनेट में हेरफेर की संभावनाओं के बीच विदेश मंत्री एसएम कृष्णा ने राहुल गांधी के मंत्रिमंडल में शामिल होने की जोरदार वकालत करते हुए कहा कि देश के सामने आ रहीं समस्याओं के समाधान के लिए यह जरूरी है।

Headline
 जयललिता की भतीजी दीपा को पोएस गार्डन स्थित आवास में नहीं घुसने दिया गया   


summary
 अन्नाद्रमुक (अम्मा) धड़े के सूत्रों ने दीपा पर हमले की बात से इनकार किया
दीपक ने जयललिता की मौत की ‘साजिश रचने के लिए’ ‘गुप्त समझौता’ किया
'मैंने स्थिति से अवगत कराने के लिए प्रधानमंत्री मोदी से समय मांगा है'

Headline
 अस्थायी नियुक्तियों में होगा 25% का इजाफा   


summary
 देश में अस्थायी नियुक्तियां बढ़ रही हैं और इस साल बैंकिंग सहित कई क्षेत्रों में इस तरह की नौकरियों में 20 से 25 प्रतिशत का इजाफा होगा।


In [ ]:
!pip install sentencepiece


In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
sequences = [
    "मैंने अपनी पूरी जिंदगी में एक हगिंगफेस कोर्स की प्रतीक्षा की है।",
       "यह कोर्स अद्भुत है!",
]
tokenizer(sequences, padding=True, truncation=True, return_tensors="tf")

In [ ]:
def tokenize_function(example):

    return tokenizer(example["headline"], example["article"], truncation=True)

In [ ]:
tokenized_datasets = dataset.map(tokenize_function, batched=True)
tokenized_datasets

Map:   0%|          | 0/13807 [00:00<?, ? examples/s]

Map:   0%|          | 0/17265 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['headline', 'summary', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 55226
    })
    test: Dataset({
        features: ['headline', 'summary', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 13807
    })
    valid: Dataset({
        features: ['headline', 'summary', 'article', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 17265
    })
})

In [ ]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")
data_collator

DataCollatorWithPadding(tokenizer=BertTokenizerFast(name_or_path='bert-base-uncased', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True), padding=True, max_length=None, pad_to_multiple_of=None, return_tensors='tf')

In [ ]:
tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "token_type_ids"],
    label_cols=["summary"],
    shuffle=True,
    collate_fn=data_collator,
    batch_size=8,
)

# tf_validation_dataset = tokenized_datasets["valid"].to_tf_dataset(
#     columns=["attention_mask", "input_ids", "token_type_ids"],
#     label_cols=["input_ids"],
#     shuffle=False,
#     collate_fn=data_collator,
#     batch_size=8,
# )

/usr/local/lib/python3.10/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


RuntimeError: ignored

In [ ]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained(checkpoint)

All PyTorch model weights were used when initializing TFBertForSequenceClassification.

Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

model.compile(
    optimizer="adam",
    loss=SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"],
)
model.fit(
    # epochs=5,
    tf_train_dataset,
    validation_data=tf_validation_dataset,
)

InvalidArgumentError: ignored